## Runnable

In [1]:
import os
os.chdir("../../")
os.getcwd()

'/home/yeonwoo/code/study/langchain_note'

In [2]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

### RunnablePassthrough
입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다.

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("{num}의 10배는?")
llm = ChatOpenAI()
chain = prompt | llm
chain

PromptTemplate(input_variables=['num'], template='{num}의 10배는?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f0836080580>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f0836081c60>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [6]:
from langchain_core.runnables import RunnablePassthrough
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

In [7]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()
runnable_chain

{
  num: RunnablePassthrough()
}
| PromptTemplate(input_variables=['num'], template='{num}의 10배는?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f083570d450>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f083570ea40>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [8]:
runnable_chain.invoke(10)

AIMessage(content='100.', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 15, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ea13abc1-4443-4b0e-94e5-4228a10c10e6-0', usage_metadata={'input_tokens': 15, 'output_tokens': 2, 'total_tokens': 17})

#### RunnablePassthrough.assign()
- 입력 값으로 들어온 값의 key/value 쌍과 새롭게 할당된 key/value 쌍을 합칩니다.

In [9]:
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

### RunnableParallel

In [10]:
from langchain_core.runnables import RunnableParallel

runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달합니다. 이는 입력된 데이터를 그대로 통과시키는 역할을 합니다.
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용하여, 'mult' 람다 함수를 할당합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가시킵니다.
    extra=RunnablePassthrough().assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달합니다. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더합니다.
    modified=lambda x: x["num"] + 1,
)
runnable.invoke({"num":10})

{'passed': {'num': 10}, 'extra': {'num': 10, 'mult': 30}, 'modified': 11}

In [11]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 19, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1c8f97b6-e61f-4d75-929f-b9dd1b3b7666-0', usage_metadata={'input_tokens': 19, 'output_tokens': 15, 'total_tokens': 34}),
 'area': AIMessage(content='대한민국의 면적은 약 100,363km² 입니다.', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 20, 'total_tokens': 42}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-659a575a-83bd-4ff5-a4d9-ac1c95261458-0', usage_metadata={'input_tokens': 20, 'output_tokens': 22, 'total_tokens': 42})}

### RunnableLambda
사용자 정의 함수를 맵핑할 수 있습니다.

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

'Jul-15'

In [13]:
from langchain_core.runnables import RunnableLambda
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 한국 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)

llm = ChatOpenAI(temperature=0)

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# 출력
chain.invoke(3)

'1. 김태희 - 1980년 3월 29일\n2. 박서준 - 1988년 12월 16일\n3. 송중기 - 1985년 9월 19일'

In [14]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


# 문장의 길이를 반환하는 함수입니다.
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수입니다.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)
chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 25는 30입니다.', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 22, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-85a05a4e-b3d9-4882-9dcf-d71090207366-0', usage_metadata={'input_tokens': 22, 'output_tokens': 9, 'total_tokens': 31})